# Test

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from learn import *
sys.path.insert(0, 'code')
from v17 import *
from v17 import _internal_validate_predict_best_param
sys.path.insert(0, 'Pytorch-UNet/utils')
from crf import dense_crf

In [ ]:
sz = 256
num_slice = 9

num_gpus = 2
gpu_start = 8
num_workers = 8
# device_ids = range(gpu_start, gpu_start + num_gpus)
device_ids = [4,5,8,9]
torch.cuda.set_device(gpu_start)
bs = 15

## Preprocessing

In [ ]:
datapaths = ['data/train/AOI_2_Vegas_Train', 'data/train/AOI_3_Paris_Train', 'data/train/AOI_4_Shanghai_Train', 'data/train/AOI_5_Khartoum_Train']
# !python code/v5_im-full_rgb.py preproc_test {datapaths[0]}

In [ ]:
# !python code/v5_im-full_rgb.py preproc_test {datapaths[3]}

In [ ]:
# !python code/v5_im-full_rgb.py preproc_test {datapaths[0]}
# !python code/v5_im-full_rgb.py preproc_test {datapaths[2]}

In [ ]:
model_name = 'deeplab'
data = None

In [ ]:
learn, denorm, data = learner_on_dataset(datapaths[3], bs, device_ids, num_workers, model_name='deeplab',
                                         debug=False, global_dataset=True, data=data, num_slice=num_slice, sz=sz,
                                         is_pred=False, test_only=True)
(trn_x,trn_y), (val_x,val_y) = data

## Inferencing

In [ ]:
learn.load('deeplab-vegas')

In [ ]:
%%time
preds = learn.predict().squeeze()

In [ ]:
%%time
with ThreadPoolExecutor(max_workers=8) as e:
    x, y = list(zip(*list(e.map(learn.data.val_dl.get_batch, [[o] for o in range(preds.shape[0])]))))
x, y, preds = [np.squeeze(o) for o in (x, y, preds)]
print(preds.shape)

In [ ]:
scores = None

In [ ]:
scores, idx = plot_worse_preds(x, y, preds, learn, jaccard_coef, shift=600,
                          scores=scores, is_best=0, thresh=0.5)

In [ ]:
evalfscore(datapaths[3], preds)

## Plot images after putting back

In [ ]:
fn_train = FMT_VALTEST_IMAGELIST_PATH.format(prefix='AOI_2_Vegas')
df_train = pd.read_csv(fn_train)

In [ ]:
df_train[df_train.ImageId == 'AOI_2_Vegas_img485']

In [ ]:
preds_big = _internal_validate_predict_best_param(3, slice_pred_list=[preds])
x_big = _internal_validate_predict_best_param(3, slice_pred_list=[denorm(x)])
y_big = _internal_validate_predict_best_param(3, slice_pred_list=[y])
x_big[0].shape

In [ ]:
idx = 89
data_im = zip(preds_big[idx:idx + 9], x_big[idx:idx + 9], y_big[idx:idx + 9])
plot_ims(data_im, figsize=4)

In [ ]:
idx = 96
crfed = dense_crf((x_big[idx] * 255).astype('uint8'), (preds_big[idx]), 1, 7, 1)
plt.imshow(preds_big[idx] > 0.3)
print(jaccard_coef_cuda((V(preds_big[idx]) > 0.3).float(), V(y_big[idx])).data[0])

In [ ]:
best_sxy1, best_sxy2, best_srgb = 0, 0, 0
best_jac = 0
# rows, cols = 3, 3
# sx1 = np.arange(1, , ).astype('uint8')
# eps = np.arange(1, 80, 80 / cols).astype('uint8')
# res = np.empty([rows, cols] + list(srcs.shape))
for sxy1 in np.arange(1, 10, 3):
    for sxy2 in np.arange(1, 10, 3):
        for srgb in np.arange(1, 10, 3):
            crfed = dense_crf((x_big[idx] * 255).astype('uint8'), (preds_big[idx]),
                             sxy1=sxy1, sxy2=sxy2, srgb=srgb)
            jac = jaccard_coef_cuda(V(crfed), V(y_big[idx])).data[0]
            if jac > best_jac:
                best_jac = jac
                best_sxy1, best_sxy2, best_srgb = sxy1, sxy2, srgb
                print(sxy1, sxy2, srgb, jac)
print(best_sxy1, best_sxy2, best_srgb, best_jac)

In [ ]:
crfed = dense_crf((x_big[idx] * 255).astype('uint8'), (preds_big[idx]),
                             sxy1=1, sxy2=1, srgb=1)